In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Perturbative Gadgets for Variational Quantum Algorithms  
==========================================

*Author: Simon Cichy*

In this tutorial, we will explore the application of perturbative gadgets in 
variational quantum algorithms to outgo the issue of cost function dependent
barren plateaus, as proposed in Ref.[cite]

Some context
---------------------------

If you are not familiar yet with the concept of barren plateaus, I recomend you
first check out these other tutorials: 
- https://pennylane.ai/qml/demos/tutorial_barren_plateaus.html
- https://pennylane.ai/qml/demos/tutorial_local_cost_functions.html  

As presented in the second, barren plateaus are more severe when using global
cost functions compared to local ones. 
Thinking about VQE applications, let us consider cost functions that are 
expectation values of Hamiltonians like
$$ C = \operatorname{Tr} \left[ H V(\theta) |00\ldots 0\rangle \langle 00\ldots 0| V(\theta)^\dagger\right]. $$
In some cases, it is easy to find a local cost function to substitute a global
one, which still has the same ground state. 
For instance, one can verify that the local cost function built from 
$$ H_L = \mathbb{I} - \frac{1}{n} \sum_j |0\rangle \langle 0|_j $$
has the same ground state as the global one
$$ H_G = \mathbb{I} - |00\ldots 0\rangle \langle 00\ldots 0|  $$
and that is 
$$ |\psi (\theta_{min}) \rangle =  |00\ldots 0\rangle. $$
However, it is not always so simple. 
What if we want to find the minimum eigenenergy of 
$ H = X \otimes X \otimes Y \otimes Z + Z \otimes Y \otimes X \otimes X $ ?  
It is not always (or rather almost neveer) trivial to construct a local cost 
function which has the same minimum as some other cost function of interest,
global however. 
That is where perturbative gadgets come into play, and we will see how.

The definitions
---------------
Perturbative gadgets are a common tool in adiabatic quantum computing. 
Their goal is to find a Hamiltonian with local interactions which mimics
some other Hamiltonian with more complex couplings. 
The later is the one they would ideally want to implement for their
computation, that's why it is called the computational Hamiltonian, but can not
since it is hard to implement more than few-body interactions in hardware.
This is done by increasing the dimension of the Hilbert space (i.e. the number 
of qubits) and "encoding" the computational Hamiltonian in the low-energy 
subspace of a so-called gadget Hamiltonian.
Let us now construct such a gadget Hamiltonian taylored for VQE applications.  

First, we start from a target Hamiltonian which is a linear combination of 
Pauli words, acting on $k$ qubits
$$ H^\text{targ} = \sum_i c_i h_i $$
with $ h_i = \sigma_{i,1} \otimes \sigma_{i,2} \otimes \ldots \otimes \sigma_{i,k} $
and $ \sigma_{i,j} \in \{ X, Y, Z \}$, $ c_i \in \mathbb{R}$.  
Now we construct the gadget Hamiltonian.
For each term $h_i$, we will need $k$ additional qubits qubits which we call 
auxiliary qubits, and add two terms to the Hamiltonian: 
an "unperturbed" part $h^\text{aux}_s$ and a perturbation $\lambda V_s$. 
The unperturbed part penalizes each of the newly added qubits for not being in 
the $|0\rangle$ state
$$ h^\text{aux}_i = \sum_{j=1}^k |1\rangle \langle 1|_{i,j}$$
while the perturbation part implements one of the operators in the Pauli word
$\sigma_{i,j}$ on the corresponding qubit of the computational register and a 
pair of Pauli $X$ gates on two of the auxiliary qubits
$$ V_i = \sum_{j=1}^k c_{i,j} \sigma_{i,j} \otimes X_{i,j} \otimes X_{i,(j+1) \mathrm{mod }k}. $$
In the end, 
$$ H^\text{gad} = \sum_{i} \left( h^\text{aux}_i + \lambda V_i \right) $$  
TODO: add picture of registers  
TODO: finish explanation of gadgets  
TODO: add figure about gadget analysis?

Constructing the gadget Hamiltonian
----------------------------------
Now that we have layed the definitions, let us have a look at one of these
constructions to get used to them. 
First we will have to import a few packages

In [1]:
import sys
sys.path.append('../src')
sys.path.append('src')
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from faehrmann_gadgets import NewPerturbativeGadgets

Now, let's take the example given above 
$$ H = X \otimes X \otimes Y \otimes Z + Z \otimes Y \otimes X \otimes X. $$
First we construct our target Hamiltonian.
For this, we use the Hamiltonian class from Pennylane

In [2]:
term1 = qml.operation.Tensor(qml.PauliX(0), qml.PauliX(1), qml.PauliY(2), qml.PauliZ(3))
term2 = qml.operation.Tensor(qml.PauliZ(0), qml.PauliY(1), qml.PauliX(2), qml.PauliX(3))
Hcomp = qml.Hamiltonian([1,1], [term1, term2])
print(Hcomp)

  (1) [X0 X1 Y2 Z3]
+ (1) [Z0 Y1 X2 X3]


Next, we can construct the corresponding gadget Hamiltonian. 
Using an automated module, we can use

In [4]:
gadgetizer = NewPerturbativeGadgets(perturbation_factor=1)
Hgad = gadgetizer.gadgetize(Hcomp)
print(Hgad)

  (-0.5) [Z4]
+ (-0.5) [Z5]
+ (-0.5) [Z6]
+ (-0.5) [Z7]
+ (-0.5) [Z8]
+ (-0.5) [Z9]
+ (-0.5) [Z10]
+ (-0.5) [Z11]
+ (0.5) [I4]
+ (0.5) [I5]
+ (0.5) [I6]
+ (0.5) [I7]
+ (0.5) [I8]
+ (0.5) [I9]
+ (0.5) [I10]
+ (0.5) [I11]
+ (-0.03125) [X4 X5 X0]
+ (-0.03125) [X8 X9 Z0]
+ (0.03125) [X5 X6 X1]
+ (0.03125) [X6 X7 Y2]
+ (0.03125) [X7 X4 Z3]
+ (0.03125) [X9 X10 Y1]
+ (0.03125) [X10 X11 X2]
+ (0.03125) [X11 X8 X3]


So, let's see what did we get.  
We started with 4 computational qubits (labelled 0 to 3) and two 4-body terms.
Thus we get 4 additional qubits twice (4 to 11).
The first 16 elements of our Hamiltonian correspond to the unperturbed part.
The last 8 are the perturbation. They are a little scambled, but one can 
recognize the 8 Paulis from the computational Hamiltonian on the qubits 0 to 3
and the cyclic pairwise $X$ structure on the auxiliaries. 
Indeed, there are $(X_4X_5, X_5X_6, X_6X_7, X_7X_4)$ and 
$(X_8X_9, X_9X_{10}, X_{10}X_{11}, X_{11}X_8)$.  


Training with the gadget Hamiltonian
-----------------------------------

Now that we have a little intuition on how the gadget Hamiltonian construction
works, we will use it to train. 
Classical simulation of qubit systems is expensive, so we will simplify further
to a computational Hamiltonian with a single term, and show that using the 
gadget Hamiltonian for training allows us to minimize the target Hamiltonian.  
So, let us construct the two Hamiltonians of interest

In [5]:
term1 = qml.operation.Tensor(qml.PauliX(0), qml.PauliX(1), qml.PauliY(2), qml.PauliZ(3))
Hcomp = qml.Hamiltonian([1], [term1])
Hgad = gadgetizer.gadgetize(Hcomp)

Then we need to set up our variational quantum algorithms. 
That is we choose a circuit ansatz with randomly initialized weights, 
the training cost function, the optimizer with its step size, and the number of 
optimization steps and the device to run the circuit on

In [ ]:
num_qubits = 4 + 2 * 4
init_weights = [np.pi/4] * num_qubits
shapes = qml.SimplifiedTwoDesign.shape(n_layers=num_qubits, n_wires=num_qubits)
weights = [np.random.uniform(0, np.pi, size=shape) for shape in shapes]
ansatz = qml.SimplifiedTwoDesign(initial_layer_weights=init_weights, weights=weights)
opt = qml.GradientDescentOptimizer(stepsize=0.3)
max_iter = 500
perturbation_factor = 10
dev = qml.device("default.qubit", wires=range(num_qubits))
training_cost = qml.ExpvalCost(ansatz, Hgad, dev)
monitoring_cost = qml.ExpvalCost(ansatz, Hcomp, dev)

In [ ]:
costs_lists = {}
costs_lists['training'] = []
costs_lists['monitoring'] = []
for it in range(max_iter):
    weights = opt.step(training_cost, weights)
    costs_lists['training'].append(training_cost(weights))
    costs_lists['monitoring'].append(monitoring_cost(weights))
    if (it + 1) % 50 == 0:
        print(f"Iteration = {it+1:5d} of {max_iter:5d} | " +
               "Training cost = {:12.8f} | ".format(costs_lists['training'][-1]) +
               "Monitoring cost = {:12.8f} | ".format(costs_lists['monitoring'][-1]))